In [1]:
#Written by GC -- 02/24

#This notebook contains code to generate ign dists for each NTS sheet based of the national fire database (NFDB). The overall approach here is to
#create a circular buffer around an NTS sheet, calc the ratio of fire per year to fuel pixels for this buffered area, and then multiple the the number of
#fuel pix per NTS sheet to get a spatially smoothed ign number that is based on real-world fire data.  
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import contextily as cx
import numpy as np
import matplotlib.cm as cm
import rasterio as rio
import rioxarray as rxr

In [2]:
#Read in nts sheet shapefile
df_nts=gpd.read_file("C:\\Users\\GiovanniCorti\\Downloads\\nts_snrc\\nts_snrc_250k.shp")
df_nts=df_nts.to_crs(epsg=6931) #Convert to equal-area projection

In [ ]:
#Read in NFDB data and select fires larger than 1 ha between 2010-2020. This is done to 1) implicity 
#include the impact of fire suppression and 2) remove fires that too small to be simulated by BP3+
df_pnts=gpd.read_file("C:\\Users\\GiovanniCorti\\Downloads\\NFDB_point\\NFDB_point_20220901.shp")
df_pnts=df_pnts[df_pnts['YEAR'].between(2010,2020)]
df_pnts=df_pnts[df_pnts['SIZE_HA'].ge(1)]

#Now count number of fires in each NTS sheet

#Check which NTS sheet a fire is in
df_pnts=df_pnts.to_crs(df_nts.crs)
fire_loc_ls=[]
for index, row in df_pnts.iterrows():
    inside=df_nts.geometry.contains(row.geometry)
    try:
        fire_loc_ls.append(df_nts[inside]['NTS_SNRC'].values[0])
    except:
        pass

#Aggregate and average to get mean annual fires per NTS sheet
df_numfires=pd.Series(fire_loc_ls).value_counts().to_frame()
df_numfires.index.name='NTS_SNRC'
df_numfires_large=df_nts.merge(df_numfires, on='NTS_SNRC')
df_numfires_large=df_numfires_large.rename(columns={'count':'lg_count'})
df_numfires_large['lg_count']=df_numfires_large['lg_count']/11

In [6]:
#Calculate the number of fuel pixels per NTS sheet. 
dt=df_nts.to_crs(fbp.rio.crs)

nts_list=[]
fuel_list=[]
for index, row in dt.iterrows():
    polys = gpd.GeoSeries((row.geometry))
    nts_list.append(row['NTS_SNRC'])
    try:
        clip_fbp=fbp.rio.clip(polys, from_disk=True)
        fuel=np.where((clip_fbp.values > 100) & (clip_fbp.values <150), 0, 1)
        fuel_list.append(np.sum(fuel))
    except:
        fuel_list.append(0)

dict = {'NTS_SNRC': nts_list, 'f_pix': fuel_list} 
df_nts_fpix=pd.DataFrame(dict)


In [9]:
#Buffer NTS sheet centroids to create a 150km radius circle around each sheet. This area, 
# which overlaps with circles from adjacent NTS sheets will be used for spatial smoothing
df_ignb=df_nts.copy()
df_ignb=df_ignb.to_crs(epsg='6931')
df_ignb['geometry']=df_ignb['geometry'].centroid.buffer(150000)


In [10]:
#Read in NFDB data and select fires larger than 1 ha between 2010-2020. This is done to 1) implicity 
#include the impact of fire suppression and 2) remove fires that too small to be simulated by BP3+
df_pnts=gpd.read_file("C:\\Users\\GiovanniCorti\\Downloads\\NFDB_point\\NFDB_point_20220901.shp")
df_pnts=df_pnts[df_pnts['YEAR'].between(2010,2020)]
df_pnts=df_pnts[df_pnts['SIZE_HA'].ge(1)]
#df_pnts=df_pnts.to_crs(epsg='4326')
df_pnts=df_pnts.to_crs(epsg='6931')
df_ignb['count']=0

#Check which fires fall within an NTS circle
fire_loc_ls=[]
for index, row in df_ignb.iterrows():
    inside=df_pnts['geometry'].within(row.geometry)
    num_igns=len(inside[inside==True])
    try:
        df_ignb.at[index,'count']=num_igns
    except:
        print('fail')
        pass


In [11]:
#Calc number of fuel pixels in each NTS circle (i.e., buffered sheet) 
fbp = rxr.open_rasterio("C:\\Users\\GiovanniCorti\\Downloads\\Canadian_Forest_FBP_Fuel_Types_v20191114\\fuel_layer\\FBP_FuelLayer.tif")
dt=df_ignb.to_crs(fbp.rio.crs)

nts_list=[]
fuel_list=[]
for index, row in dt.iterrows():
    polys = gpd.GeoSeries((row.geometry))
    nts_list.append(row['NTS_SNRC'])
    try:
        clip_fbp=fbp.rio.clip(polys, from_disk=True)
        fuel=np.where((clip_fbp.values < 100) | (clip_fbp.values >150), 1, 0)
        fuel_list.append(np.sum(fuel))
    except:
        fuel_list.append(0)

dict = {'NTS_SNRC': nts_list, 'f_pix_buff': fuel_list} 
dt=pd.DataFrame(dict)

#Calc fire to fuel pix ratio for NTS circle and multiply by num fuel pix per sheet. 
df_buff=df_ignb.merge(dt, on='NTS_SNRC')
df_buff['ratio']=df_buff['count']/df_buff['f_pix_buff']
df_buff=df_buff.merge(df_nts_fpix, on='NTS_SNRC',how='left')
df_buff['ign_num']=(df_buff['f_pix']*df_buff['ratio'])/11
df_ign_num=df_nts.merge(df_buff[['NTS_SNRC','ign_num']], on='NTS_SNRC', how='left')
df_ign_num=df_ign_num.to_crs(epsg=3857)


In [31]:
#Save to shape file
#df_ign_num.to_file("C:/Users/GiovanniCorti/Documents/Wildfire/ign_v2.shp")

In [81]:
#Create ign dist .csvs for each NTS sheet
for index, row in df_plot.iterrows():
    num=np.round(row['ign_num'],2)
    vals=np.round(np.modf(num),2)
    ign_num_ls=[]
    per_ls=[]
    
    if num==0:
        #r1=[0,1.0]
        ign_num_ls.append(0)
        per_ls.append(100)
    elif np.isnan(num):
        pass
    else:
        ign_num_ls.extend((int(vals[1]),int(vals[1]+1)))
        per_ls.extend((100*np.round(1-vals[0],2),100*np.round(vals[0],2)))
    dict = {'ign_per_it': ign_num_ls, 'pct': per_ls} 
    dt=pd.DataFrame(dict)
    #Write .csvs to Y drive
    dt.to_csv("Y:client-data\\demo_projects\\climate85\\Working_data\\ign_dist\\ign_dist_"+row['NTS_SNRC']+'.csv',index=False)